In [1]:
import os
os.chdir('../../../../data')
!ls

FAOSTAT_A-S_E               area_reductions
FAOSTAT_T-Z_E               livestock_2010
GAEZ_2015                   ppp_2020_1km_Aggregated.tif
SPAM_2010                   s41597-021-01115-2.pdf
admin_polygons              transport_sample.csv
admin_polygons.gpkg


In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [10]:
item = 'maize'
country = 'kenya'
year = 2020

## Areas

In [103]:
admin = gpd.read_file('admin_polygons.gpkg')

In [104]:
maize_areas = pd.read_csv('area_reductions/maize_areas.csv').drop(['system:index', '.geo'], axis=1)
banana_areas = pd.read_csv('area_reductions/banana_areas.csv').drop(['system:index', '.geo'], axis=1)
soybean_areas = pd.read_csv('area_reductions/soybean_areas.csv').drop(['system:index', '.geo'], axis=1)
cattle_counts = pd.read_csv('area_reductions/cattle_areas.csv').drop(['system:index', '.geo'], axis=1)
coffee_areas = pd.read_csv('area_reductions/coffee_areas.csv').drop(['system:index', '.geo'], axis=1)
pop_counts = pd.read_csv('area_reductions/pop_areas.csv').drop(['system:index', '.geo'], axis=1)

In [8]:
maize_areas.head()

,ID,admin,admin_name,country,iso3,latitude,longitude,maize_sum
0,AFG.1_1,admin1,Badakhshan,Afghanistan,AFG,37.032895,71.458977,1650.748531
1,AFG.2_1,admin1,Badghis,Afghanistan,AFG,35.139751,63.752696,3088.383955
2,AFG.3_1,admin1,Baghlan,Afghanistan,AFG,35.705362,68.713877,2212.028742
3,AFG.4_1,admin1,Balkh,Afghanistan,AFG,36.648028,67.134957,11855.269809
4,AFG.6_1,admin1,Daykundi,Afghanistan,AFG,33.754922,66.183993,320.851629


## Production, trade

In [15]:
prod = pd.read_csv('FAOSTAT_A-S_E/Production_Crops_Livestock_E_All_Data_(Normalized)/Production_Crops_Livestock_E_All_Data_(Normalized).csv',
                   encoding='latin1')
prod_area_codes = pd.read_csv('FAOSTAT_A-S_E/Production_Crops_Livestock_E_All_Data_(Normalized)/Production_Crops_Livestock_E_AreaCodes.csv',
                               encoding='latin1')
prod_flags = pd.read_csv('FAOSTAT_A-S_E/Production_Crops_Livestock_E_All_Data_(Normalized)/Production_Crops_Livestock_E_Flags.csv',
                          encoding='latin1')
prod_ItemCodes = pd.read_csv('FAOSTAT_A-S_E/Production_Crops_Livestock_E_All_Data_(Normalized)/Production_Crops_Livestock_E_ItemCodes.csv',
                              encoding='latin1')


In [16]:
prod.head()

,Area Code,Area Code (M49),Area,Item Code,Item Code (CPC),Item,Element Code,Element,Year Code,Year,Unit,Value,Flag
0,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,1975,1975,ha,0.0,E
1,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,1976,1976,ha,5900.0,E
2,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,1977,1977,ha,6000.0,E
3,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,1978,1978,ha,6000.0,E
4,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,1979,1979,ha,6000.0,E


In [17]:
prod = prod[['Area', 'Item', 'Element', 'Year', 'Unit', 'Value']]

In [18]:
prod = prod[(prod['Year']==year) & (prod['Item']=='Maize (corn)') & (prod['Element']=='Production')].reset_index(drop=True)

In [19]:
prod.head()

,Area,Item,Element,Year,Unit,Value
0,Afghanistan,Maize (corn),Production,2020,tonnes,271776.0
1,Albania,Maize (corn),Production,2020,tonnes,399125.0
2,Algeria,Maize (corn),Production,2020,tonnes,3221.0
3,Angola,Maize (corn),Production,2020,tonnes,2972177.0
4,Antigua and Barbuda,Maize (corn),Production,2020,tonnes,11.4


In [20]:
mat = pd.read_csv('FAOSTAT_T-Z_E/Trade_DetailedTradeMatrix_E_All_Data_(Normalized)/Trade_DetailedTradeMatrix_E_All_Data_(Normalized).csv',
                encoding='latin1')

In [31]:
mat = mat[(mat['Year']==year) & (mat['Item']=='Maize (corn)') 
        & (mat['Unit']=='tonnes') & (mat['Element']=='Import Quantity')][[
    'Reporter Countries', 'Partner Countries', 'Item', 'Element', 'Year', 'Unit', 'Value']].reset_index(drop=True)

In [32]:
mat.head()

,Reporter Countries,Partner Countries,Item,Element,Year,Unit,Value
0,Burundi,Argentina,Maize (corn),Import Quantity,2020,tonnes,0.40
1,Djibouti,Argentina,Maize (corn),Import Quantity,2020,tonnes,4796.58
2,Cambodia,Argentina,Maize (corn),Import Quantity,2020,tonnes,188349.90
3,Bangladesh,Argentina,Maize (corn),Import Quantity,2020,tonnes,107485.00
4,Sierra Leone,Argentina,Maize (corn),Import Quantity,2020,tonnes,468.00


In [48]:
mat[(mat['Reporter Countries'].isin(['Philippines', 'Argentina'])) & (mat['Partner Countries'].isin(['Philippines', 'Argentina']))]

,Reporter Countries,Partner Countries,Item,Element,Year,Unit,Value
53,Philippines,Argentina,Maize (corn),Import Quantity,2020,tonnes,6733.42
1658,Argentina,Philippines,Maize (corn),Import Quantity,2020,tonnes,0.01


In [67]:
imports = mat.groupby('Reporter Countries').agg({'Item': 'first', 'Element': 'first',
                                                 'Year': 'first', 'Unit': 'first', 'Value': 'sum'}).reset_index()
imports = imports.rename(columns={'Reporter Countries': 'Area'})

In [68]:
imports.head()

,Area,Item,Element,Year,Unit,Value
0,Albania,Maize (corn),Import Quantity,2020,tonnes,83401.96
1,Algeria,Maize (corn),Import Quantity,2020,tonnes,5010403.34
2,Antigua and Barbuda,Maize (corn),Import Quantity,2020,tonnes,83.89
3,Argentina,Maize (corn),Import Quantity,2020,tonnes,3771.62
4,Armenia,Maize (corn),Import Quantity,2020,tonnes,71466.95


In [69]:
exports = mat.groupby('Partner Countries').agg({'Item': 'first', 'Element': 'first',
                                                 'Year': 'first', 'Unit': 'first', 'Value': 'sum'}).reset_index()
exports['Element'] = 'Export Quantity'
exports = exports.rename(columns={'Partner Countries': 'Area'})

In [70]:
exports.head()

,Area,Item,Element,Year,Unit,Value
0,Afghanistan,Maize (corn),Export Quantity,2020,tonnes,55.00
1,Algeria,Maize (corn),Export Quantity,2020,tonnes,5.00
2,Angola,Maize (corn),Export Quantity,2020,tonnes,84.00
3,Antigua and Barbuda,Maize (corn),Export Quantity,2020,tonnes,105.00
4,Argentina,Maize (corn),Export Quantity,2020,tonnes,35359598.65


In [93]:
df = pd.concat([prod, imports, exports], axis=0, ignore_index=True)
df = pd.pivot(df, index=['Area','Item', 'Year', 'Unit'], columns = 'Element',values = 'Value').reset_index()
df = df.fillna(0)
df['Consumption'] = df['Production'] + df['Import Quantity'] - df['Export Quantity']

#### next steps
- clean trade data
- clean counry names to join with trade data
- apply proportions
- compare with available sub national data

In [97]:
df[df['Consumption']<0]

Element,Area,Item,Year,Unit,Export Quantity,Import Quantity,Production,Consumption
6,Antigua and Barbuda,Maize (corn),2020,tonnes,105.00,83.89,11.40,-9.71
14,Bahamas,Maize (corn),2020,tonnes,36469.81,1025.81,663.32,-34780.68
17,Barbados,Maize (corn),2020,tonnes,111575.97,31529.39,86.00,-79960.58
43,"China, Macao SAR",Maize (corn),2020,tonnes,19.52,0.00,0.00,-19.52
177,Singapore,Maize (corn),2020,tonnes,45356.42,9828.63,0.00,-35527.79
201,Tokelau,Maize (corn),2020,tonnes,0.01,0.00,0.00,-0.01
202,Tonga,Maize (corn),2020,tonnes,0.66,0.41,0.00,-0.25


In [99]:
mat[mat['Partner Countries'].isin(['Antigua and Barbuda', 'Bahamas', 'Barbados', 'China, Macao SAR', 'Singapore', 'Tokelau', 'Tonga'])]

,Reporter Countries,Partner Countries,Item,Element,Year,Unit,Value
30,Saint Lucia,Antigua and Barbuda,Maize (corn),Import Quantity,2020,tonnes,0.00
56,Grenada,Barbados,Maize (corn),Import Quantity,2020,tonnes,39.68
58,Saint Kitts and Nevis,Barbados,Maize (corn),Import Quantity,2020,tonnes,3.81
135,Venezuela (Bolivarian Republic of),Bahamas,Maize (corn),Import Quantity,2020,tonnes,36469.81
140,Venezuela (Bolivarian Republic of),Barbados,Maize (corn),Import Quantity,2020,tonnes,111532.48
209,United Kingdom of Great Britain and Northern I...,Antigua and Barbuda,Maize (corn),Import Quantity,2020,tonnes,24.00
397,Netherlands,Antigua and Barbuda,Maize (corn),Import Quantity,2020,tonnes,81.00
575,Brunei Darussalam,Singapore,Maize (corn),Import Quantity,2020,tonnes,0.58
768,Seychelles,Singapore,Maize (corn),Import Quantity,2020,tonnes,991.58
907,Zambia,Tokelau,Maize (corn),Import Quantity,2020,tonnes,0.01


In [110]:
df['Area'].unique()

array(['Afghanistan', 'Africa', 'Albania', 'Algeria', 'Americas',
       'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Asia',
       'Australia', 'Australia and New Zealand', 'Austria', 'Azerbaijan',
       'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus',
       'Belgium', 'Belize', 'Benin', 'Bhutan',
       'Bolivia (Plurinational State of)', 'Bosnia and Herzegovina',
       'Botswana', 'Brazil', 'Brunei Darussalam', 'Bulgaria',
       'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon',
       'Canada', 'Caribbean', 'Central African Republic',
       'Central America', 'Central Asia', 'Chad', 'Chile', 'China',
       'China, Hong Kong SAR', 'China, Macao SAR',
       'China, Taiwan Province of', 'China, mainland', 'Colombia',
       'Comoros', 'Congo', 'Cook Islands', 'Costa Rica', 'Croatia',
       'Cuba', 'Cyprus', 'Czechia', "Côte d'Ivoire",
       "Democratic People's Republic of Korea",
       'Democratic Republic of the Congo', 'Denmark', 'Djib

In [109]:
admin['country'].unique()

array(['United Kingdom', 'Afghanistan', 'Angola', 'Åland', 'Albania',
       'Andorra', 'United Arab Emirates', 'Argentina', 'Armenia',
       'American Samoa', 'French Southern Territories',
       'Antigua and Barbuda', 'Australia', 'Austria', 'Azerbaijan',
       'Burundi', 'Belgium', 'Benin', 'Bonaire, Sint Eustatius and Saba',
       'Burkina Faso', 'Bangladesh', 'Bulgaria', 'Bahrain', 'Bahamas',
       'Bosnia and Herzegovina', 'Belarus', 'Belize', 'Bermuda',
       'Bolivia', 'Brazil', 'Barbados', 'Brunei', 'Bhutan', 'Botswana',
       'Central African Republic', 'Canada', 'Switzerland', 'Chile',
       'China', "Côte d'Ivoire", 'Cameroon',
       'Democratic Republic of the Congo', 'Republic of Congo',
       'Colombia', 'Comoros', 'Cape Verde', 'Costa Rica', 'Cuba',
       'Cayman Islands', 'Cyprus', 'Czech Republic', 'Germany',
       'Djibouti', 'Dominica', 'Denmark', 'Dominican Republic', 'Algeria',
       'Ecuador', 'Egypt', 'Eritrea', 'Western Sahara', 'Spain',
       'Es